## prerequisites:

- Beautifulsoup
- selenium
- pandas
- webdriver manager
- openpyxl

In [54]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import urllib3
import re
from bs4 import BeautifulSoup
import csv
http=urllib3.PoolManager()
import requests
import json

# Scraping all the links of hackathons

Set ScrollNumber variable in code to any number to gather more links of hackathons hosted on this site

In [8]:
driver = webdriver.Chrome(ChromeDriverManager().install())

#The website that consists of all the hackathons and their links
driver.get("https://unstop.com/hackathons?filters=,all,all,all&types=teamsize,payment,eligible,oppstatus")

#for just testing purposes the number of scrolls are set to 3, we can increase it any number depending on the 
# hackathons we want to extract contact information for.
ScrollNumber = 3
for i in range(1,ScrollNumber):
    driver.execute_script("window.scrollTo(1,50000)")
    time.sleep(5)

# DS.html has the html code for the infinite scrolling page    
file = open('DS.html', 'w')
file.write(driver.page_source)
file.close()

driver.close()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 96.0.4664
[WDM] - Get LATEST chromedriver version for 96.0.4664 google-chrome
[WDM] - Driver [/home/rohit/.wdm/drivers/chromedriver/linux64/96.0.4664.45/chromedriver] found in cache


## Scrape the HTML code and extract links for all the hackathons and output to excel

In [82]:
def scrape(file_name='DS.html'):
    contact_link_list = []
    contacts_json_list = []
    
    data = open(file_name,'r')
    soup = BeautifulSoup(data, 'html.parser')
    for link in soup.find_all('a', {'class': re.compile(r"listing order-\d id-[0-9]+ full_banner ng-star-inserted", re.IGNORECASE)}):
        contact_link, contacts_json = extract_contacts('https://unstop.com'+link.get('href'))
        contact_link_list.append(contact_link)
        contacts_json_list.append(contacts_json)
    return contact_link_list, contacts_json_list

# Using the API to extract contacts:
def extract_contacts(link):
    
    contact_link = ''
    contacts_json = ''
    non_contact_link = ''
    
    
    last_bit= link.split('/')[-1].split('-')[-1]
    response_API = requests.get('https://unstop.com/api/public/competition/'+last_bit)
    data = response_API.text
    parse_json = json.loads(data)
    if 'data' in parse_json and 'competition' in parse_json['data']:
        if 'contacts' in parse_json['data']['competition']:
            contact_link = link
            contacts_json = parse_json['data']['competition']['contacts']
            return contact_link, contacts_json
        else:
            non_contact_link = link
            contacts_json = ''
            return non_contact_link, contacts_json

contact_link_list, contacts_json_list = scrape()

df = pd.DataFrame({'hackathon links': links, 'Contact_information': contacts_json_list, 'length':[len(i) for i in contacts_json_list]})
contacts_df = df[df['length']!=0][['hackathon links','Contact_information']]
non_contacts_df = df[df['length']==0][['hackathon links','Contact_information']]

contacts_df.to_excel('contacts.xlsx', sheet_name='sheet1', index=False)
non_contacts_df.to_excel('non_contacts.xlsx', sheet_name='sheet1', index=False)


In [ ]:
## Using the api method instead of scrapping method(so the code below is not useful for now)

# ## Trying to figure out how to extract contact information:

# ### Stuck here: [created a forum link](https://forum.webscraper.io/t/extract-contact-information-from-website-but-stuck/10543)

# import os
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# import time
# from webdriver_manager.chrome import ChromeDriverManager
# import pandas as pd

# driver = webdriver.Chrome(ChromeDriverManager().install())

# driver.get("https://unstop.com/hackathon/weave-the-web-powered-by-edot-solutions-techyon-2022-padre-conceicao-college-of-engineering-salcete-goa-462438")
# stages = driver.find_elements_by_xpath("/html/body/div[1]/main/app-public-competition/div/div[2]/div[2]/div/div[1]/div[3]/div/app-dates-and-contacts/section/div/div[2]/ul")
# print(stages)


# ## For contacts extract all of the content under the class "contact ng-star-inserted"